In [15]:
import numpy as np
import pandas as pd
from PIL import Image
import os
import cv2
import numpy as np

from config import *


In [16]:
#img_batch_paths = os.listdir("../sample_imgs") # /new_sample_imgs_3.npy
# this makes a list that is out of order (not the order they were written in, 0-23)

In [9]:
#Takes a minute or so to load
frame_df = pd.read_hdf(FRAME_DATAFRAME_PATH, key='master_frames')
frame_df.head()

#sample_df = sample_dataframe(frame_df, sample_size=GOLD_SET_NUM_FRAMES)

,vid_name,vid_path,child_id,frame,unix_time,age_days,nose_conf,wrist_conf,face_openpose,hand_openpose
0,A_20130531_0818_01,/scratch/groups/mcfrank/Home_Headcam_new/A_201...,A,0,2013-05-31,260,0.0,0.0,False,False
1,A_20130531_0818_01,/scratch/groups/mcfrank/Home_Headcam_new/A_201...,A,1,2013-05-31,260,0.0,0.0,False,False
2,A_20130531_0818_01,/scratch/groups/mcfrank/Home_Headcam_new/A_201...,A,2,2013-05-31,260,0.0,0.0,False,False
3,A_20130531_0818_01,/scratch/groups/mcfrank/Home_Headcam_new/A_201...,A,3,2013-05-31,260,0.0,0.0,False,False
4,A_20130531_0818_01,/scratch/groups/mcfrank/Home_Headcam_new/A_201...,A,4,2013-05-31,260,0.0,0.0,False,False


In [11]:
sample_df = pd.read_json(SAMPLE_DATAFRAME_PATH)
print(sample_df.info())
sample_df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24000 entries, 0 to 9999
Data columns (total 17 columns):
level_0                 24000 non-null int64
face_openpose           24000 non-null int64
face_openpose_nose      24000 non-null int64
face_present            24000 non-null int64
frame                   24000 non-null int64
hand_openpose           24000 non-null int64
hand_openpose_wrist     12000 non-null float64
hand_present            24000 non-null int64
index                   24000 non-null int64
vid_name                24000 non-null object
vid_path                24000 non-null object
face_keypoints          24000 non-null object
pose_keypoints          24000 non-null object
hand_left_keypoints     24000 non-null object
hand_right_keypoints    24000 non-null object
face_keypoints_tuple    24000 non-null object
pose_keypoints_tuple    24000 non-null object
dtypes: float64(1), int64(8), object(8)
memory usage: 3.3+ MB
None


,level_0,face_openpose,face_openpose_nose,face_present,frame,hand_openpose,hand_openpose_wrist,hand_present,index,vid_name,vid_path,face_keypoints,pose_keypoints,hand_left_keypoints,hand_right_keypoints,face_keypoints_tuple,pose_keypoints_tuple
0,0,0,1,1,3515,0,NaN,1,0,S_20141112_2426_03.mp4,/scratch/groups/mcfrank/Home_Headcam_new/Samca...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0.471533, 0.0980919, 0.37534799999999996, 0....","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..."
1,1,0,0,0,4925,0,NaN,1,1,S_20131127_1310_04.mp4,/scratch/groups/mcfrank/Home_Headcam_new/Samca...,[],[],[],[],"[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..."
10,10,1,1,1,8785,1,NaN,1,10005,S_20141228_2611_08.mp4,/scratch/groups/mcfrank/Home_Headcam_new/Samca...,"[[0.281036, 0.524218, 0.0931592, 0.281939, 0.5...","[[0.295931, 0.558211, 0.7980659999999999, 0.37...","[[0.319839, 0.901076, 0.47536000000000006, 0.3...","[[0.183019, 0.7725029999999999, 0.0520404, 0.1...","[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..."
100,100,0,0,1,14425,0,NaN,0,10087,S_20130619_0802_03.mp4,/scratch/groups/mcfrank/Home_Headcam_new/Samca...,[],[],[],[],"[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..."
1000,1000,0,0,1,1470,0,NaN,0,10898,S_20141115_2429_01.mp4,/scratch/groups/mcfrank/Home_Headcam_new/Samca...,[],[],[],[],"[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..."


In [60]:
# originally I just read in gold_sample.json and assumed it was in the same order as sample_df was returned
# from sample_dataframe (and same order as the new_sample_imgs_N.npy files), but it doesn't look like it was
# -- so now let's sort the same way sample_df is sorted in sample_dataframe
sample_df = sample_df.sort_values(by=['vid_name', 'frame']).reset_index(drop=True)
sample_df.head() # will output in this order in frames2 

,level_0,face_openpose,face_openpose_nose,face_present,frame,hand_openpose,hand_openpose_wrist,hand_present,index,vid_name,vid_path,face_keypoints,pose_keypoints,hand_left_keypoints,hand_right_keypoints,face_keypoints_tuple,pose_keypoints_tuple
0,2553,1,1,0,525,1,1.0,0,1496,A_20130531_0818_01.mp4,/scratch/groups/mcfrank/Home_Headcam_new/Alice...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0.610381, 0.424906, 0.39134599999999997, 0.6...","[[0.6413559999999999, 0.515645, 0.0103926, 0.6...","[[0.606552, 0.5423319999999999, 0.01349, 0.607...","[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[[[0.6103489999999999, 0.394781, 0.209307, 0.6..."
1,5824,0,0,0,1795,0,0.0,0,444,A_20130531_0818_01.mp4,/scratch/groups/mcfrank/Home_Headcam_new/Alice...,[],[],[],[],"[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..."
2,8609,0,0,0,2980,0,0.0,1,6947,A_20130531_0818_01.mp4,/scratch/groups/mcfrank/Home_Headcam_new/Alice...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 0, 0, 0.265374, 0.0707734, 0.145416, 0.31...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..."
3,11689,0,1,0,3335,0,0.0,1,9719,A_20130531_0818_01.mp4,/scratch/groups/mcfrank/Home_Headcam_new/Alice...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0.0224281, 0.147013, 0.0528444, 0.0489932, 0...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..."
4,1494,0,0,0,6670,0,0.0,0,11341,A_20130531_0818_01.mp4,/scratch/groups/mcfrank/Home_Headcam_new/Alice...,[],[],[],[],"[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..."


In [61]:
#imgs = np.load("../sample_imgs/new_sample_imgs_3.npy")
batch_size = 1000

In [42]:
#cv2.imshow('Frame', imgs[0])
sample_df.frame.describe()
row = sample_df.loc[0, ['vid_name','frame']]
row['vid_name'].split('.')[0] +'_'+ str(row['frame'])#sample_df.level_0.describe()
str(0) +'-' + row['vid_name'] +'-'+ str(row['frame']) + '.jpg'

'0-S_20141112_2426_03.mp4-3515.jpg'

In [19]:
img = Image.fromarray(imgs[0], 'RGB')
img.save('test.png') # 150kb
img.save('test.jpg') # 19.5kb - let's do that
#img.show()

In [57]:
img_batch_paths = []
for i in range(24):
    new_path = 'new_sample_imgs_' + str(i) + '.npy'
    img_batch_paths.append(new_path)

print(img_batch_paths) # order they were created...

['new_sample_imgs_0.npy', 'new_sample_imgs_1.npy', 'new_sample_imgs_2.npy', 'new_sample_imgs_3.npy', 'new_sample_imgs_4.npy', 'new_sample_imgs_5.npy', 'new_sample_imgs_6.npy', 'new_sample_imgs_7.npy', 'new_sample_imgs_8.npy', 'new_sample_imgs_9.npy', 'new_sample_imgs_10.npy', 'new_sample_imgs_11.npy', 'new_sample_imgs_12.npy', 'new_sample_imgs_13.npy', 'new_sample_imgs_14.npy', 'new_sample_imgs_15.npy', 'new_sample_imgs_16.npy', 'new_sample_imgs_17.npy', 'new_sample_imgs_18.npy', 'new_sample_imgs_19.npy', 'new_sample_imgs_20.npy', 'new_sample_imgs_21.npy', 'new_sample_imgs_22.npy', 'new_sample_imgs_23.npy']


In [62]:
i = 0
for imgs_batch_path in img_batch_paths:
    imgs = np.load(os.path.join("../sample_imgs",imgs_batch_path))
    idx = 0
    while idx < len(imgs):
        print(f'Frame {i} of {GOLD_SET_NUM_FRAMES}')
        row = sample_df.loc[i, ['vid_name','frame']]
        fname = str(i) +'-' + row['vid_name'] +'-'+ str(row['frame']) + '.jpg' # .split('.')[0] # drop .mp4
        #['vid_name','frame']
        img = Image.fromarray(imgs[idx], 'RGB')
        img.save('frames2/'+fname)
        idx += 1
        i += 1
        #img.show()

Frame 0 of 24000
Frame 1 of 24000
Frame 2 of 24000
Frame 3 of 24000
Frame 4 of 24000
Frame 5 of 24000
Frame 6 of 24000
Frame 7 of 24000
Frame 8 of 24000
Frame 9 of 24000
Frame 10 of 24000
Frame 11 of 24000
Frame 12 of 24000
Frame 13 of 24000
Frame 14 of 24000
Frame 15 of 24000
Frame 16 of 24000
Frame 17 of 24000
Frame 18 of 24000
Frame 19 of 24000
Frame 20 of 24000
Frame 21 of 24000
Frame 22 of 24000
Frame 23 of 24000
Frame 24 of 24000
Frame 25 of 24000
Frame 26 of 24000
Frame 27 of 24000
Frame 28 of 24000
Frame 29 of 24000
Frame 30 of 24000
Frame 31 of 24000
Frame 32 of 24000
Frame 33 of 24000
Frame 34 of 24000
Frame 35 of 24000
Frame 36 of 24000
Frame 37 of 24000
Frame 38 of 24000
Frame 39 of 24000
Frame 40 of 24000
Frame 41 of 24000
Frame 42 of 24000
Frame 43 of 24000
Frame 44 of 24000
Frame 45 of 24000
Frame 46 of 24000
Frame 47 of 24000
Frame 48 of 24000
Frame 49 of 24000
Frame 50 of 24000
Frame 51 of 24000
Frame 52 of 24000
Frame 53 of 24000
Frame 54 of 24000
Frame 55 of 24000
Fr

Frame 443 of 24000
Frame 444 of 24000
Frame 445 of 24000
Frame 446 of 24000
Frame 447 of 24000
Frame 448 of 24000
Frame 449 of 24000
Frame 450 of 24000
Frame 451 of 24000
Frame 452 of 24000
Frame 453 of 24000
Frame 454 of 24000
Frame 455 of 24000
Frame 456 of 24000
Frame 457 of 24000
Frame 458 of 24000
Frame 459 of 24000
Frame 460 of 24000
Frame 461 of 24000
Frame 462 of 24000
Frame 463 of 24000
Frame 464 of 24000
Frame 465 of 24000
Frame 466 of 24000
Frame 467 of 24000
Frame 468 of 24000
Frame 469 of 24000
Frame 470 of 24000
Frame 471 of 24000
Frame 472 of 24000
Frame 473 of 24000
Frame 474 of 24000
Frame 475 of 24000
Frame 476 of 24000
Frame 477 of 24000
Frame 478 of 24000
Frame 479 of 24000
Frame 480 of 24000
Frame 481 of 24000
Frame 482 of 24000
Frame 483 of 24000
Frame 484 of 24000
Frame 485 of 24000
Frame 486 of 24000
Frame 487 of 24000
Frame 488 of 24000
Frame 489 of 24000
Frame 490 of 24000
Frame 491 of 24000
Frame 492 of 24000
Frame 493 of 24000
Frame 494 of 24000
Frame 495 of

Frame 878 of 24000
Frame 879 of 24000
Frame 880 of 24000
Frame 881 of 24000
Frame 882 of 24000
Frame 883 of 24000
Frame 884 of 24000
Frame 885 of 24000
Frame 886 of 24000
Frame 887 of 24000
Frame 888 of 24000
Frame 889 of 24000
Frame 890 of 24000
Frame 891 of 24000
Frame 892 of 24000
Frame 893 of 24000
Frame 894 of 24000
Frame 895 of 24000
Frame 896 of 24000
Frame 897 of 24000
Frame 898 of 24000
Frame 899 of 24000
Frame 900 of 24000
Frame 901 of 24000
Frame 902 of 24000
Frame 903 of 24000
Frame 904 of 24000
Frame 905 of 24000
Frame 906 of 24000
Frame 907 of 24000
Frame 908 of 24000
Frame 909 of 24000
Frame 910 of 24000
Frame 911 of 24000
Frame 912 of 24000
Frame 913 of 24000
Frame 914 of 24000
Frame 915 of 24000
Frame 916 of 24000
Frame 917 of 24000
Frame 918 of 24000
Frame 919 of 24000
Frame 920 of 24000
Frame 921 of 24000
Frame 922 of 24000
Frame 923 of 24000
Frame 924 of 24000
Frame 925 of 24000
Frame 926 of 24000
Frame 927 of 24000
Frame 928 of 24000
Frame 929 of 24000
Frame 930 of

Frame 1295 of 24000
Frame 1296 of 24000
Frame 1297 of 24000
Frame 1298 of 24000
Frame 1299 of 24000
Frame 1300 of 24000
Frame 1301 of 24000
Frame 1302 of 24000
Frame 1303 of 24000
Frame 1304 of 24000
Frame 1305 of 24000
Frame 1306 of 24000
Frame 1307 of 24000
Frame 1308 of 24000
Frame 1309 of 24000
Frame 1310 of 24000
Frame 1311 of 24000
Frame 1312 of 24000
Frame 1313 of 24000
Frame 1314 of 24000
Frame 1315 of 24000
Frame 1316 of 24000
Frame 1317 of 24000
Frame 1318 of 24000
Frame 1319 of 24000
Frame 1320 of 24000
Frame 1321 of 24000
Frame 1322 of 24000
Frame 1323 of 24000
Frame 1324 of 24000
Frame 1325 of 24000
Frame 1326 of 24000
Frame 1327 of 24000
Frame 1328 of 24000
Frame 1329 of 24000
Frame 1330 of 24000
Frame 1331 of 24000
Frame 1332 of 24000
Frame 1333 of 24000
Frame 1334 of 24000
Frame 1335 of 24000
Frame 1336 of 24000
Frame 1337 of 24000
Frame 1338 of 24000
Frame 1339 of 24000
Frame 1340 of 24000
Frame 1341 of 24000
Frame 1342 of 24000
Frame 1343 of 24000
Frame 1344 of 24000


Frame 1714 of 24000
Frame 1715 of 24000
Frame 1716 of 24000
Frame 1717 of 24000
Frame 1718 of 24000
Frame 1719 of 24000
Frame 1720 of 24000
Frame 1721 of 24000
Frame 1722 of 24000
Frame 1723 of 24000
Frame 1724 of 24000
Frame 1725 of 24000
Frame 1726 of 24000
Frame 1727 of 24000
Frame 1728 of 24000
Frame 1729 of 24000
Frame 1730 of 24000
Frame 1731 of 24000
Frame 1732 of 24000
Frame 1733 of 24000
Frame 1734 of 24000
Frame 1735 of 24000
Frame 1736 of 24000
Frame 1737 of 24000
Frame 1738 of 24000
Frame 1739 of 24000
Frame 1740 of 24000
Frame 1741 of 24000
Frame 1742 of 24000
Frame 1743 of 24000
Frame 1744 of 24000
Frame 1745 of 24000
Frame 1746 of 24000
Frame 1747 of 24000
Frame 1748 of 24000
Frame 1749 of 24000
Frame 1750 of 24000
Frame 1751 of 24000
Frame 1752 of 24000
Frame 1753 of 24000
Frame 1754 of 24000
Frame 1755 of 24000
Frame 1756 of 24000
Frame 1757 of 24000
Frame 1758 of 24000
Frame 1759 of 24000
Frame 1760 of 24000
Frame 1761 of 24000
Frame 1762 of 24000
Frame 1763 of 24000


...

Frame 22363 of 24000
Frame 22364 of 24000
Frame 22365 of 24000
Frame 22366 of 24000
Frame 22367 of 24000
Frame 22368 of 24000
Frame 22369 of 24000
Frame 22370 of 24000
Frame 22371 of 24000
Frame 22372 of 24000
Frame 22373 of 24000
Frame 22374 of 24000
Frame 22375 of 24000
Frame 22376 of 24000
Frame 22377 of 24000
Frame 22378 of 24000
Frame 22379 of 24000
Frame 22380 of 24000
Frame 22381 of 24000
Frame 22382 of 24000
Frame 22383 of 24000
Frame 22384 of 24000
Frame 22385 of 24000
Frame 22386 of 24000
Frame 22387 of 24000
Frame 22388 of 24000
Frame 22389 of 24000
Frame 22390 of 24000
Frame 22391 of 24000
Frame 22392 of 24000
Frame 22393 of 24000
Frame 22394 of 24000
Frame 22395 of 24000
Frame 22396 of 24000
Frame 22397 of 24000
Frame 22398 of 24000
Frame 22399 of 24000
Frame 22400 of 24000
Frame 22401 of 24000
Frame 22402 of 24000
Frame 22403 of 24000
Frame 22404 of 24000
Frame 22405 of 24000
Frame 22406 of 24000
Frame 22407 of 24000
Frame 22408 of 24000
Frame 22409 of 24000
Frame 22410 o

Frame 22762 of 24000
Frame 22763 of 24000
Frame 22764 of 24000
Frame 22765 of 24000
Frame 22766 of 24000
Frame 22767 of 24000
Frame 22768 of 24000
Frame 22769 of 24000
Frame 22770 of 24000
Frame 22771 of 24000
Frame 22772 of 24000
Frame 22773 of 24000
Frame 22774 of 24000
Frame 22775 of 24000
Frame 22776 of 24000
Frame 22777 of 24000
Frame 22778 of 24000
Frame 22779 of 24000
Frame 22780 of 24000
Frame 22781 of 24000
Frame 22782 of 24000
Frame 22783 of 24000
Frame 22784 of 24000
Frame 22785 of 24000
Frame 22786 of 24000
Frame 22787 of 24000
Frame 22788 of 24000
Frame 22789 of 24000
Frame 22790 of 24000
Frame 22791 of 24000
Frame 22792 of 24000
Frame 22793 of 24000
Frame 22794 of 24000
Frame 22795 of 24000
Frame 22796 of 24000
Frame 22797 of 24000
Frame 22798 of 24000
Frame 22799 of 24000
Frame 22800 of 24000
Frame 22801 of 24000
Frame 22802 of 24000
Frame 22803 of 24000
Frame 22804 of 24000
Frame 22805 of 24000
Frame 22806 of 24000
Frame 22807 of 24000
Frame 22808 of 24000
Frame 22809 o

Frame 23153 of 24000
Frame 23154 of 24000
Frame 23155 of 24000
Frame 23156 of 24000
Frame 23157 of 24000
Frame 23158 of 24000
Frame 23159 of 24000
Frame 23160 of 24000
Frame 23161 of 24000
Frame 23162 of 24000
Frame 23163 of 24000
Frame 23164 of 24000
Frame 23165 of 24000
Frame 23166 of 24000
Frame 23167 of 24000
Frame 23168 of 24000
Frame 23169 of 24000
Frame 23170 of 24000
Frame 23171 of 24000
Frame 23172 of 24000
Frame 23173 of 24000
Frame 23174 of 24000
Frame 23175 of 24000
Frame 23176 of 24000
Frame 23177 of 24000
Frame 23178 of 24000
Frame 23179 of 24000
Frame 23180 of 24000
Frame 23181 of 24000
Frame 23182 of 24000
Frame 23183 of 24000
Frame 23184 of 24000
Frame 23185 of 24000
Frame 23186 of 24000
Frame 23187 of 24000
Frame 23188 of 24000
Frame 23189 of 24000
Frame 23190 of 24000
Frame 23191 of 24000
Frame 23192 of 24000
Frame 23193 of 24000
Frame 23194 of 24000
Frame 23195 of 24000
Frame 23196 of 24000
Frame 23197 of 24000
Frame 23198 of 24000
Frame 23199 of 24000
Frame 23200 o

Frame 23558 of 24000
Frame 23559 of 24000
Frame 23560 of 24000
Frame 23561 of 24000
Frame 23562 of 24000
Frame 23563 of 24000
Frame 23564 of 24000
Frame 23565 of 24000
Frame 23566 of 24000
Frame 23567 of 24000
Frame 23568 of 24000
Frame 23569 of 24000
Frame 23570 of 24000
Frame 23571 of 24000
Frame 23572 of 24000
Frame 23573 of 24000
Frame 23574 of 24000
Frame 23575 of 24000
Frame 23576 of 24000
Frame 23577 of 24000
Frame 23578 of 24000
Frame 23579 of 24000
Frame 23580 of 24000
Frame 23581 of 24000
Frame 23582 of 24000
Frame 23583 of 24000
Frame 23584 of 24000
Frame 23585 of 24000
Frame 23586 of 24000
Frame 23587 of 24000
Frame 23588 of 24000
Frame 23589 of 24000
Frame 23590 of 24000
Frame 23591 of 24000
Frame 23592 of 24000
Frame 23593 of 24000
Frame 23594 of 24000
Frame 23595 of 24000
Frame 23596 of 24000
Frame 23597 of 24000
Frame 23598 of 24000
Frame 23599 of 24000
Frame 23600 of 24000
Frame 23601 of 24000
Frame 23602 of 24000
Frame 23603 of 24000
Frame 23604 of 24000
Frame 23605 o

Frame 23973 of 24000
Frame 23974 of 24000
Frame 23975 of 24000
Frame 23976 of 24000
Frame 23977 of 24000
Frame 23978 of 24000
Frame 23979 of 24000
Frame 23980 of 24000
Frame 23981 of 24000
Frame 23982 of 24000
Frame 23983 of 24000
Frame 23984 of 24000
Frame 23985 of 24000
Frame 23986 of 24000
Frame 23987 of 24000
Frame 23988 of 24000
Frame 23989 of 24000
Frame 23990 of 24000
Frame 23991 of 24000
Frame 23992 of 24000
Frame 23993 of 24000
Frame 23994 of 24000
Frame 23995 of 24000
Frame 23996 of 24000
Frame 23997 of 24000
Frame 23998 of 24000
Frame 23999 of 24000


In [63]:
frame_fnames = os.listdir("frames2")
for i in range(len(frame_fnames)):
    frame_fnames[i] = 'http://langcog.stanford.edu/expts/saycam/frames/' + frame_fnames[i]

df = pd.DataFrame(columns=['img_src'])
df['img_src'] = frame_fnames 
df.to_csv('frame_urls2.csv')